In [1]:
import numpy as np
import pandas as pd

## Loading the Dataset

In [167]:
in_file = 'titanic_data.csv'
full_data = pd.read_csv(in_file)

display(full_data.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Description of Columns

- Survived: Outcome of survival (0 = No; 1 = Yes)

- Pclass: Socio-economic class (1 = Upper class; 2 = Middle class; 3 = Lower class)
- Name: Name of passenger
- Sex: Sex of the passenger
- Age: Age of the passenger (Some entries contain NaN)
- SibSp: Number of siblings and spouses of the passenger aboard
- Parch: Number of parents and children of the passenger aboard
- Ticket: Ticket number of the passenger
- Fare: Fare paid by the passenger
- Cabin Cabin number of the passenger (Some entries contain NaN)
- Embarked: Port of embarkation of the passenger (C = Cherbourg; Q = Queenstown; S = Southampton)


In [141]:
result = full_data["Survived"]
data = full_data.drop("Survived", axis=1)
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Initial Hypothesis : All Died (As Majority Died)

Prediction = series of 0's with length equal to input data set

In [18]:
pred = np.zeros(len(data), dtype=int)

### Accuracy Function

In [19]:
def accuracy_score(real_data, pred_data):
    if(len(real_data)==len(pred_data)):
        print "Accuracy Score : {}".format((real_data==pred_data).mean()*100)

accuracy_score(result, pred)

Accuracy Score : 61.6161616162


## Hypothesis 1: All male died and all female survived

In [58]:
pred = full_data["Sex"].map({'male':0, 'female':1})
accuracy_score(result, pred)

Accuracy Score : 78.6756453423


## Hypothesis 3: Female with higher class survived

In [78]:
# full_data['Female_Class'] = ((full_data["Pclass"] == 1) | (full_data["Pclass"] == 2)) & (full_data["Sex"] == 'female')
# test_data_2 = full_data["Female_Class"]

for i in range(1,4):

    full_data['Female_Class'] = ((full_data["Pclass"] == i) & (full_data["Sex"] == 'female'))
    pred = full_data["Female_Class"]
    print "Accuracy for : (Female with class {})".format(i)
    accuracy_score(result, pred)

Accuracy for : (Female with class 1)
Accuracy Score : 71.492704826
Accuracy for : (Female with class 2)
Accuracy Score : 68.7991021324
Accuracy for : (Female with class 3)
Accuracy Score : 61.6161616162


Accuracy not increasing, so for now lets suppose all female survived and see if high class men survived or not.

## Hypothesis 4: Men with higher class survived

In [68]:
for i in range(1,4):
    
    full_data['Men_Class'] = ((full_data["Pclass"] == i) & (full_data["Sex"] == 'male')) | (full_data["Sex"] == 'female')
    pred = full_data["Men_Class"]
    print "Accuracy for : (Men with class {}) or (Female)".format(i)
    accuracy_score(result, pred)

Accuracy for : (Men with class 1) or (Female)
Accuracy Score : 75.0841750842
Accuracy for : (Men with class 2) or (Female)
Accuracy Score : 70.3703703704
Accuracy for : (Men with class 3) or (Female)
Accuracy Score : 50.2805836139


## Hypothesis 5: Higher class survived

In [82]:
for i in range(1,4):

    full_data["Class_Survival"] = full_data["Pclass"] == i
    pred = full_data["Class_Survival"]
    print "Accuracy for : (Class {})".format(i)
    accuracy_score(result, pred)

Accuracy for : (Class 1)
Accuracy Score : 67.9012345679
Accuracy for : (Class 2)
Accuracy Score : 60.4938271605
Accuracy for : (Class 3)
Accuracy Score : 33.2210998878


## Hypothesis 6: Men with Lower age survived

In [109]:
# full_data.Age = full_data.Age.fillna(0)
# full_data.Age = full_data.Age.astype(int)


full_data["Age_Survival"] = (full_data["Sex"] == 'female') | ((full_data["Sex"]=='male') & (full_data["Age"] <= 10.0))
pred = full_data["Age_Survival"]
print "Accuracy for : (Age <10)"
accuracy_score(result, pred)

Accuracy for : (Age <10)
Accuracy Score : 79.2368125701


## Hypothesis 7: High Fares Men survived

In [124]:
full_data["Fare_Survival"] = (((full_data["Fare"] >= 400.0) | (full_data["Age"] <= 10.0)) & (full_data["Sex"]=='male')) | (full_data["Sex"]=='female')
pred = full_data["Fare_Survival"]
print "Accuracy for : (Fare >= 400)"
accuracy_score(result, pred)

Accuracy for : (Fare >= 400)
Accuracy Score : 79.4612794613


# Final Hypothesis: W/O using ML

In [121]:
def predictions(data):
    """ Model with multiple features. Makes a prediction with an accuracy of at least 80%. """
    
    test_data_final = []
    for _, passenger in data.iterrows():
        
        if(passenger['Sex'] == 'female'):
            if(passenger['SibSp']>=3 or (passenger['Pclass']==3 and passenger['Parch']>=2)):
                test_data_final.append(0)
            else:
                test_data_final.append(1)
        else:
            if(passenger['Age']<=10.0 or passenger['Fare']>=400.0 or (passenger['Fare']>=120.0 and passenger['Fare']<=150.0)):
                test_data_final.append(1)
            else:
                test_data_final.append(0)
            
        pass
    
    # Return our predictions
    return pd.Series(test_data_final)

# Make the predictions
test_data_final = predictions(data)

In [123]:
accuracy_score(result, test_data_final)

Accuracy Score : 81.3692480359


We are still at ~81.5% accuracy. 

## Trying ML

## Decision Tree

In [173]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score as score

In [185]:
test = full_data.fillna(0)
result = test["Survived"]
data = test.drop("Survived", axis=1)
data = data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]]
# data = data.dropna(axis=0)
data.Sex = data.Sex.map({'male': 0, 'female': 1})
X_train, X_test, y_train, y_test = train_test_split(data, result, test_size=0.2, random_state=1)
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [187]:
y_pred = clf.predict(X_test)
print score(y_test, y_pred)


0.7653631284916201


## Random Forest

In [218]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=6,random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [219]:
y_pred = clf.predict(X_test)
print score(y_test, y_pred)

0.7927170868347339
